In [547]:
import polars as pl
import pandas as pd
import os
import json
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
from datetime import timezone

start_date = datetime(2023, 7, 10, 0, 0, 0).replace(tzinfo=timezone.utc)
end_date = datetime(2023, 7, 10, 23, 59, 59).replace(tzinfo=timezone.utc)

data = {"date": [],
        "system name": [],
        "variant": [],
        "CO2_raw": [],
        "CO2_compensated": [],
        "CO2_filtered": [],
        "sht45_humidity": [],
        "bme280_humidity": [],
        "bme280_pressure": [],
        "sht45_temperature": [],
        "bme280_temperature": [],
        "chamber_temperature": [],
        "gas_bottle_id": []}


def two_point_calibration(measured_values, true_values):
    # Check if input lists have length 2
    if len(measured_values) != 2 or len(true_values) != 2:
        raise ValueError("Both measured_values and true_values must have length 2")

    # Calculate calibration parameters (slope and intercept)
    # 
    slope = (true_values[1] - true_values[0]) / (measured_values[1] - measured_values[0])
    # y_true = m * y_meas + t
    intercept = true_values[0] - slope * measured_values[0]

    return slope, intercept

In [548]:
df_gas = pl.read_csv("../data/averaged_gases.csv")
df_gas.head()

Bottle_ID,CO2_dry
i64,f64
1,798.115438
2,807.238809
3,391.345476
4,399.530674
5,416.594961


In [549]:
quickflow_directory = "/Users/patrickaigner/Documents/PROJECTS/ACROPOLIS/Software/quickflow/notebooks/acropolis/"
df_quickflow = pl.scan_parquet(os.path.join(quickflow_directory, "measurements.parquet")) 

In [550]:
# loop through the quickflow dataset and sort it into dataframe
df_q = df_quickflow.select(pl.col("creation_timestamp","sensor_name","value")).sort("creation_timestamp").filter(pl.col("creation_timestamp").is_between(start_date, end_date)).collect()

for payload in df_q.iter_rows():
    creation_timestamp = payload[0]
    sensor_name = payload[1]
    entry = json.loads(payload[2])
    
    if entry["variant"] in ("measurement","calibration"):
        # base
        data["date"].append(creation_timestamp)
        data["system name"].append(sensor_name)
        data["variant"].append(entry["variant"])
        #variant measurement & calibrtion
        data["CO2_raw"].append(entry["data"]["raw"])
        data["CO2_compensated"].append(entry["data"]["compensated"])
        data["CO2_filtered"].append(entry["data"]["compensated"])
        data["sht45_humidity"].append(entry["data"]["sht45_humidity"])
        data["bme280_humidity"].append(entry["data"]["bme280_humidity"])
        data["bme280_pressure"].append(entry["data"]["bme280_pressure"])
        data["sht45_temperature"].append(entry["data"]["sht45_temperature"])
        data["bme280_temperature"].append(entry["data"]["bme280_temperature"])
        data["chamber_temperature"].append(entry["data"]["chamber_temperature"])
        if entry["variant"] == "calibration":
                data["gas_bottle_id"].append(entry["data"]["gas_bottle_id"])
        else:
                data["gas_bottle_id"].append(None)
        
df = pl.DataFrame(data)
df.head()    

date,system name,variant,CO2_raw,CO2_compensated,CO2_filtered,sht45_humidity,bme280_humidity,bme280_pressure,sht45_temperature,bme280_temperature,chamber_temperature,gas_bottle_id
"datetime[μs, UTC]",str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
2023-07-10 00:00:00.740 UTC,"""tum-esm-midcos…","""measurement""",408.4,460.5,460.5,33.77,34.23,936.3,28.98,28.9,30.2,null
2023-07-10 00:00:01.200 UTC,"""tum-esm-midcos…","""measurement""",0.0,0.0,0.0,36.85,34.64,941.3,27.34,27.41,28.8,null
2023-07-10 00:00:01.420 UTC,"""tum-esm-midcos…","""measurement""",375.3,431.1,431.1,33.85,32.5,937.09,28.93,28.95,30.6,null
2023-07-10 00:00:03.690 UTC,"""tum-esm-midcos…","""measurement""",411.0,461.6,461.6,32.59,33.39,943.31,29.68,29.2,31.0,null
2023-07-10 00:00:05.310 UTC,"""tum-esm-midcos…","""measurement""",409.8,458.2,458.2,32.02,31.51,945.15,29.89,29.52,31.0,null


# Picarro data

In [551]:
df_p = pd.read_csv("../data/picarro/CFADS2343-20230710-000010Z-DataLog_User.dat",sep='\s+')
df_p["datetime"] = pd.to_datetime((df_p['DATE'] + ' ' + df_p['TIME']))

# correct Picarro data
pic_slope = 1.0009341797161153 
pic_intercept = 0.32176843460007376
df_p["CO2_corr"] = df_p["CO2_dry"] * pic_slope + pic_intercept 

df_p.head()

,DATE,TIME,FRAC_DAYS_SINCE_JAN1,FRAC_HRS_SINCE_JAN1,JULIAN_DAYS,EPOCH_TIME,ALARM_STATUS,INST_STATUS,CavityPressure,CavityTemp,...,CO2_dry,h2o_reported,ch4_base,ch4_pzt_std,co2_base,co2_pzt_std,wlm1_offset,wlm2_offset,datetime,CO2_corr
0,2023-07-10,00:00:00.323,190.000004,4560.000090,191.000004,1.688947e+09,0,963,139.998062,44.999809,...,453.393610,1.765826,1181.722668,91.326854,1084.834526,70.594089,-0.074958,-0.045083,2023-07-10 00:00:00.323,454.138930
1,2023-07-10,00:00:01.872,190.000022,4560.000520,191.000022,1.688947e+09,0,963,140.014389,44.999809,...,453.393610,1.765826,1181.670743,91.326854,1085.023989,70.594089,-0.074953,-0.045093,2023-07-10 00:00:01.872,454.138930
2,2023-07-10,00:00:02.783,190.000032,4560.000773,191.000032,1.688947e+09,0,963,140.045125,44.999809,...,453.393610,1.765826,1181.670743,91.326854,1085.035482,70.594089,-0.074943,-0.045093,2023-07-10 00:00:02.783,454.138930
3,2023-07-10,00:00:03.292,190.000038,4560.000915,191.000038,1.688947e+09,0,963,140.044525,44.999809,...,453.366484,1.765067,1181.670743,91.326854,1085.035482,70.594089,-0.074943,-0.045093,2023-07-10 00:00:03.292,454.111778
4,2023-07-10,00:00:04.832,190.000056,4560.001342,191.000056,1.688947e+09,0,963,140.047283,44.999870,...,453.366484,1.765067,1181.608542,83.169072,1084.916155,55.066913,-0.074953,-0.045086,2023-07-10 00:00:04.832,454.111778


# System data

In [552]:
def avg_meas_bottle(df, bottle_id = 0):
    
    df_temp = df.filter(pl.col("gas_bottle_id") == str(bottle_id))
    
    if len(df_temp) > 60:
        df_temp = df_temp.filter(pl.col("CO2_filtered") > 0)
        avg = df_temp["CO2_filtered"][(int(len(df_f)*0.5)):(int(len(df_f)*0.95))].mean()
    else:
        df_temp = df_temp.filter(pl.col("CO2_filtered") > 0)
        avg = df_temp["CO2_filtered"][(int(len(df_f)*0.3)):(int(len(df_f)*0.95))].mean()
        
    #sns.lineplot(data=df_temp,x='date',y="CO2_compensated")
    return avg


In [559]:
# System 1 has bottles 6 & 2
bottles = [2,6] #400ppm, 800ppm
system = 1

def correct_vaisala_data(df, system, bottles):
    bottles_meas = [avg_meas_bottle(df, bottle_id = bottles[0]),avg_meas_bottle(df, bottle_id = bottles[1])]
    bottles_true = [df_gas.filter(pl.col("Bottle_ID")== bottles[0])["CO2_dry"][0],df_gas.filter(pl.col("Bottle_ID")== bottles[1])["CO2_dry"][0]]
    
    print(bottles_meas, bottles_true)

    slope, intercept = two_point_calibration(bottles_meas, bottles_true)

    df_meas = df.filter(pl.col("variant") == "measurement").filter(pl.col("system name") == f"tum-esm-midcost-raspi-{system}")
    df_meas = df_meas.filter(pl.col("CO2_filtered") > 0).filter(pl.col("CO2_filtered") < 550)
    df_meas = df_meas.sort("date").groupby_dynamic("date",every='10m').agg(pl.all().exclude('date').mean())
    df_meas = df_meas.with_columns(pl.col("CO2_filtered") * slope + intercept)
    
    return df_meas

[412.99199999999996, 814.0160000000002] [401.5412941876083, 807.2388088294385]
[390.65200000000004, 788.9079999999999] [399.5306738030804, 817.9944589683803]
[402.86, 802.1360000000001] [399.57209790624563, 809.3100835262264]
[394.1400000000001, 789.1720000000001] [406.676269659521, 808.3055093557002]
[385.01599999999996, 782.5399999999998] [403.08825054200275, 814.6655503422389]
[402.07599999999996, 788.044] [403.0484176551189, 802.1964319732738]
[413.12, 827.4760000000001] [400.67101965388713, 816.2521645761756]
[427.95200000000006, 817.0] [417.4174905349762, 806.7391610200393]
[414.02799999999996, 813.6800000000003] [406.53149559549007, 805.5490240326694]
[412.99199999999996, 805.6279999999999] [416.5949607593189, 805.7284689396154]


In [581]:
# correct Vaisala data
sns.lineplot(data=correct_vaisala_data(df, 1, [6,2]),x='date',y="CO2_filtered", label = "1")
sns.lineplot(data=correct_vaisala_data(df, 2, [4,25]),x='date',y="CO2_filtered", label = "2") 
sns.lineplot(data=correct_vaisala_data(df, 3, [27,14]),x='date',y="CO2_filtered", label = "3")
sns.lineplot(data=correct_vaisala_data(df, 5, [22,35]),x='date',y="CO2_filtered", label = "5") #
sns.lineplot(data=correct_vaisala_data(df, 6, [7,38]),x='date',y="CO2_filtered", label = "6")
sns.lineplot(data=correct_vaisala_data(df, 7, [26,37]),x='date',y="CO2_filtered", label = "7") 
sns.lineplot(data=correct_vaisala_data(df, 8, [30,39]),x='date',y="CO2_filtered", label = "8")
sns.lineplot(data=correct_vaisala_data(df, 9, [8,33]),x='date',y="CO2_filtered", label = "9")
sns.lineplot(data=correct_vaisala_data(df, 10, [36,9]),x='date',y="CO2_filtered", label = "10")
sns.lineplot(data=correct_vaisala_data(df, 11, [5,34]),x='date',y="CO2_filtered", label = "11")
#sns.lineplot(data=df_p,x='datetime',y="CO2_corr",label="Picarro")

# set axes labels
plt.xlabel('Time')
plt.ylabel('CO2')
plt.xticks(rotation=45)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d.%m %H:%M'))

# show the plot
plt.show()

In [576]:
#df.filter(pl.col("variant") == "calibration").filter(pl.col("system name") == f"tum-esm-midcost-raspi-11").groupby("gas_bottle_id").head()

gas_bottle_id,date,system name,variant,CO2_raw,CO2_compensated,CO2_filtered,sht45_humidity,bme280_humidity,bme280_pressure,sht45_temperature,bme280_temperature,chamber_temperature
str,"datetime[μs, UTC]",str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""34""",2023-07-10 12:20:54.060 UTC,"""tum-esm-midcos…","""calibration""",423.7,474.2,474.2,11.36,11.92,956.18,37.22,37.18,33.6
"""34""",2023-07-10 12:21:04.080 UTC,"""tum-esm-midcos…","""calibration""",663.4,742.3,742.3,1.91,5.85,956.54,37.24,37.19,33.6
"""34""",2023-07-10 12:21:14.080 UTC,"""tum-esm-midcos…","""calibration""",695.3,778.5,778.5,0.84,4.72,956.76,37.29,37.24,33.6
"""34""",2023-07-10 12:21:24.050 UTC,"""tum-esm-midcos…","""calibration""",705.9,790.4,790.4,0.54,4.28,955.51,37.35,37.29,33.7
"""34""",2023-07-10 12:21:34.090 UTC,"""tum-esm-midcos…","""calibration""",718.1,805.5,805.5,0.33,4.05,954.0,37.38,37.33,33.6
"""5""",2023-07-10 12:00:43.070 UTC,"""tum-esm-midcos…","""calibration""",371.9,418.6,418.6,35.95,37.89,897.48,31.46,31.48,32.6
"""5""",2023-07-10 12:00:53.090 UTC,"""tum-esm-midcos…","""calibration""",380.4,457.0,457.0,14.55,19.71,956.57,31.47,31.46,32.6
"""5""",2023-07-10 12:01:03.110 UTC,"""tum-esm-midcos…","""calibration""",376.8,417.6,417.6,4.48,12.39,955.95,31.46,31.46,32.6
"""5""",2023-07-10 12:01:13.100 UTC,"""tum-esm-midcos…","""calibration""",362.6,402.4,402.4,2.9,10.23,954.94,31.47,31.48,32.6
